In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization import quantity_support
from astropy import constants as const
import math
from scipy import stats
import pandas as pd
import matplotlib.cm as cm
import matplotlib as mpl
import pandas as pd

In [2]:
# wavelength_band: which band you're observing in
# wavelength: wavelength where PSF is diffraction limited
# t_exp: exposure time
# tel_diameter: diameter of telescope
# magnitude: magnitude of star in wavelength_band

def ast_err(wavelength_band, wavelength, t_exp, tel_diameter, target_star_magnitude):
    aperature = np.pi * (tel_diameter.to(u.cm)/2)**2
    t_exp = t_exp.to(u.s)
    FWHM = ((wavelength.to(u.m) / tel_diameter.to(u.m)) * u.rad).to(u.arcsec)
    if wavelength_band == 'U':
        phi_U = 756.1*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_U = (0.06*u.micron).to(u.AA)
        zero_photons_U = phi_U * t_exp * aperature * delta_lambda_U
        number_of_photons_U = (10**(-target_star_magnitude/2.5)) * zero_photons_U
        SNR_U = np.sqrt(number_of_photons_U)
        return FWHM / ((2*np.sqrt(2)) * SNR_U)
    if wavelength_band == 'B':
        phi_B = 1392.6*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_B = (0.09*u.micron).to(u.AA)
        zero_photons_B = phi_B * t_exp * aperature * delta_lambda_B
        number_of_photons_B = (10**(-target_star_magnitude/2.5)) * zero_photons_B
        SNR_B = np.sqrt(number_of_photons_B)
        return FWHM / ((2*np.sqrt(2)) * SNR_B)
    if wavelength_band == 'V':
        phi_V = 995.5*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_V = (0.085*u.micron).to(u.AA)
        zero_photons_V = phi_V * t_exp * aperature * delta_lambda_V
        number_of_photons_V = (10**(-target_star_magnitude/2.5)) * zero_photons_V
        SNR_V = np.sqrt(number_of_photons_V)
        return FWHM / ((2*np.sqrt(2)) * SNR_V)
    if wavelength_band == 'R':
        phi_R = 702.0*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_R = (0.15*u.micron).to(u.AA)
        zero_photons_R = phi_R * t_exp * aperature * delta_lambda_R
        number_of_photons_R = (10**(-target_star_magnitude/2.5)) * zero_photons_R
        SNR_R = np.sqrt(number_of_photons_R)
        return FWHM / ((2*np.sqrt(2)) * SNR_R)
    if wavelength_band == 'I':
        phi_I = 452.0*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_I = (0.15*u.micron).to(u.AA)
        zero_photons_I = phi_I * t_exp * aperature * delta_lambda_I
        number_of_photons_I = (10**(-target_star_magnitude/2.5)) * zero_photons_I
        SNR_I = np.sqrt(number_of_photons_I)
        return FWHM / ((2*np.sqrt(2)) * SNR_I)
    if wavelength_band == 'J':
        phi_J = 193.1*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_J = (0.26*u.micron).to(u.AA)
        zero_photons_J = phi_J * t_exp * aperature * delta_lambda_J
        number_of_photons_J = (10**(-target_star_magnitude/2.5)) * zero_photons_J
        SNR_J = np.sqrt(number_of_photons_J)
        return FWHM / ((2*np.sqrt(2)) * SNR_J)
    if wavelength_band == 'H':
        phi_H = 93.3*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_H = (0.29*u.micron).to(u.AA)
        zero_photons_H = phi_H * t_exp * aperature * delta_lambda_H
        number_of_photons_H = (10**(-target_star_magnitude/2.5)) * zero_photons_H
        SNR_H = np.sqrt(number_of_photons_H)
        return FWHM / ((2*np.sqrt(2)) * SNR_H)
    if wavelength_band == 'K':
        phi_K = 43.6*(1/u.s)*(1/(u.cm**2))*(1/u.AA)
        delta_lambda_K = (0.41*u.micron).to(u.AA)
        zero_photons_K = phi_K * t_exp * aperature * delta_lambda_K
        number_of_photons_K = (10**(-target_star_magnitude/2.5)) * zero_photons_K
        SNR_K = np.sqrt(number_of_photons_K)
        return FWHM / ((2*np.sqrt(2)) * SNR_K)

# create an arry of magnitudes for background stars: background_star_magnitudes

def total_ast_err(wavelength_band, wavelength, t_exp, tel_diameter, target_star_magnitude, background_star_magnitudes):
    target_star_err = ast_err(wavelength_band, wavelength, t_exp, tel_diameter, target_star_magnitude)
    ref_star_err = []
    for i in range(len(background_star_magnitudes)):
        individual_ref_star_err = ast_err(wavelength_band, wavelength, t_exp, tel_diameter, i)
        ref_star_err.append(individual_ref_star_err)
# number of stars at magnitude times the detector size 
    return np.sqrt((target_star_err)**2 + sum(1/(ref_star_err)**2)**-1)


In [7]:
# read in all of the latitude files

models_by_lat = ['lat_eight_model', 'lat_eighty_model', 'lat_fifteen_model', 'lat_fifty_model',
              'lat_five_model', 'lat_forty_model', 'lat_four_model', 'lat_neg_eight_model',
              'lat_neg_eighty_model', 'lat_neg_fifteen_model', 'lat_neg_fifty_model', 'lat_neg_five_model',
              'lat_neg_forty_model', 'lat_neg_four_model', 'lat_neg_nine_model', 'lat_neg_one_model',
              'lat_neg_seven_model', 'lat_neg_seventy_model', 'lat_neg_six_model', 'lat_neg_sixty_model',
              'lat_neg_ten_model', 'lat_neg_thirty_model', 'lat_neg_three_model', 'lat_neg_twenty_model',
              'lat_neg_twentyfive_model', 'lat_neg_two_model', 'lat_nine_model', 'lat_one_model', 'lat_seven_model',
              'lat_seventy_model', 'lat_six_model', 'lat_sixty_model', 'lat_ten_model', 'lat_thirty_model',
              'lat_three_model', 'lat_twenty_five_model', 'lat_twenty_model', 'lat_two_model', 'lat_zero_model']
              
              
              
all_csv = [pd.read_csv(i, usecols=['longitude', 'latitude', 'sigma_0', 'sigma_0 errors', 'alpha_1', 'alpha_1 errors', 'alpha_2', 'alpha_2 errors', 'V_b', 'V_b errors', 's', 's errors']) for i in models_by_lat] # This will give you a list of your dfs


full_df = pd.concat(all_csv, axis=0, ignore_index = True)

full_df

,longitude,latitude,sigma_0,sigma_0 errors,alpha_1,alpha_1 errors,alpha_2,alpha_2 errors,V_b,V_b errors,s,s errors
0,0.0,8.0,3.055260e+05,"(1099.2728876430192, 1033.059045133472)",0.487186,"(0.001291195735148687, 0.0013437679339605713)",0.101033,"(0.0009951070543450513, 0.0010212401835602986)",18.523231,"(0.0054757627825381405, 0.005091452296753118)",9.941294,"(0.196136280138127, 0.08936661714076877)"
1,5.0,8.0,3.350073e+05,"(42477.97842746842, 57871.991684539884)",0.446178,"(0.0019784534270856358, 0.00204646967922123)",0.440926,"(0.0026042565537174323, 0.0023031453226690646)",18.969181,"(0.1599262485650641, 0.18936830929650128)",0.480731,"(0.03806731486390208, 0.058293964325999204)"
2,10.0,8.0,1.957164e+05,"(62295.18193867912, 40410.26198329375)",0.385633,"(0.0013111958381392674, 0.0012168980634278959)",0.382722,"(0.0016288577056516873, 0.0013791976860259014)",19.806514,"(0.497344124306796, 0.2402792840458048)",2.007031,"(0.30809577836766233, 0.3721979520687497)"
3,15.0,8.0,5.567227e+04,"(10389.083218845197, 14748.635686035166)",0.402935,"(0.0019486909119596496, 0.002190737511257479)",0.402105,"(0.002738788310776785, 0.0017679330306067942)",19.103721,"(0.26584168309873135, 0.27136794443794443)",1.623511,"(0.2895710790185555, 0.3600279569093243)"
4,20.0,8.0,1.238057e+06,"(209618.3236733782, 573346.7300364494)",0.367306,"(0.003464226120359404, 0.0036481201045930334)",0.360867,"(0.0034565862404201675, 0.00295061401250557)",16.989812,"(0.49377483287544877, 0.3576096562445983)",0.114120,"(0.014643521047411623, 0.005378908868802476)"
...,...,...,...,...,...,...,...,...,...,...,...,...
852,330.0,0.0,8.830886e+03,"(1379.9865275097736, 1981.5370032821556)",0.581551,"(0.014961945565213908, 0.01702522111324034)",0.127017,"(0.0065220109336457666, 0.0052770794727020065)",12.229293,"(0.2369901646166106, 0.2686730688037198)",0.231160,"(0.01204010669618455, 0.014069866361405142)"
853,340.0,0.0,1.372186e+06,"(209337.31043704483, 170995.05345181725)",0.796677,"(0.034054175412190024, 0.043800349285498696)",0.001766,"(0.002720576258030398, 0.004317325215599371)",12.279563,"(0.6023208650093075, 0.4879537660825264)",0.091900,"(0.005683605521185159, 0.005278626749351714)"
854,345.0,0.0,2.724541e+04,"(3599.254267305205, 3287.524193137073)",0.418436,"(0.013122810523472028, 0.01250931955928325)",0.095014,"(0.00706503417493097, 0.009399279613903577)",15.748231,"(0.20075555018438251, 0.1977911868120259)",0.584717,"(0.06990294658525864, 0.10514874599955581)"
855,350.0,0.0,3.726739e+03,"(500.73190171393253, 567.2291365855822)",0.451525,"(0.028529724460802708, 0.03283777063289445)",0.150334,"(0.0058011112507470874, 0.0068997769746193816)",13.536137,"(0.4843492558604279, 0.46091939587426545)",0.623901,"(0.13133796288939892, 0.20430136723400039)"


In [8]:
# sigma_v function

def sigma_v(theta, V_magnitudes):
    sigma_0, alpha_1, alpha_2, V_b, s = theta
    value = sigma_0 * (1/(10**(-s*alpha_1*(V_magnitudes-V_b)) + 10**(-s*alpha_2*(V_magnitudes-V_b)))) ** (1/s)
    return value

In [20]:
magnitudes = np.linspace(3, 20, 18)
# print(magnitudes)

asterometric_error = ast_err(wavelength_band = 'V', wavelength = 1 * u.micron, t_exp = 60 * u.s, tel_diameter = 6 * u.m, target_star_magnitude = magnitudes)


theta = (full_df['sigma_0'].iloc[0], full_df['alpha_1'].iloc[0], full_df['alpha_2'].iloc[0], full_df['V_b'].iloc[0], full_df['s'].iloc[0])

number_of_stars = sigma_v(theta, magnitudes)

print(np.where(number_of_stars > 1), 'np.where')

number_of_stars = number_of_stars[np.where(number_of_stars > 1)]

print(sum(number_of_stars))

asterometric_error = asterometric_error[np.where(number_of_stars > 1)]

print(asterometric_error)

error_at_each_mag = (asterometric_error)**2 * number_of_stars

print(error_at_each_mag)

total_ast_r = sum(error_at_each_mag)

print(total_ast_r, 'total reference star astrometric error at 0, 8')



(array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),) np.where
1023399.5051302891
[1.27710610e-08 2.02407676e-08 3.20794547e-08 5.08425094e-08
 8.05799471e-08 1.27710610e-07 2.02407676e-07 3.20794547e-07
 5.08425094e-07 8.05799471e-07 1.27710610e-06 2.02407676e-06
 3.20794547e-06] arcsec
[3.72160039e-16 2.87022467e-15 2.21361478e-14 1.70721493e-13
 1.31666216e-12 1.01545460e-11 7.83153086e-11 6.03994269e-10
 4.65820902e-09 3.59256855e-08 2.76799360e-07 1.39646672e-06
 4.43307260e-06] arcsec2
6.147616548136593e-06 arcsec2 total reference star astrometric error at 0, 8


In [37]:
# integrate 

from scipy.integrate import quad

def sigma_v_integrand(x, sigma_0, alpha_1, alpha_2, V_b, s):
#     sigma_0, alpha_1, alpha_2, V_b, s = theta_first
    value = 10 ** (0.4 * (x-V_b)) * (sigma_0 * (1/(10**(-s*alpha_1*(x-V_b)) + 10**(-s*alpha_2*(x-V_b)))) ** (1/s))
    return value

sigma_0, alpha_1, alpha_2, V_b, s = full_df['sigma_0'].iloc[0], full_df['alpha_1'].iloc[0], full_df['alpha_2'].iloc[0], full_df['V_b'].iloc[0], full_df['s'].iloc[0]

integrate_sigma_v = quad(sigma_v_integrand, 1, 20, args=(sigma_0, alpha_1, alpha_2, V_b, s))

integrate_sigma_v

for index, row in full_df.iterrows():
    sigma_0, alpha_1, alpha_2, V_b, s = full_df['sigma_0'].iloc[index], full_df['alpha_1'].iloc[index], full_df['alpha_2'].iloc[index], full_df['V_b'].iloc[index], full_df['s'].iloc[index]
    integrate_sigma_v = quad(sigma_v_integrand, 1, 20, args=(sigma_0, alpha_1, alpha_2, V_b, s))
    print(integrate_sigma_v / full_df['sigma_0'].iloc[index])

[4.36711560e+00 3.90013236e-13]
[9.01628991e-01 1.02728370e-14]
[5.56041324e-01 3.69759708e-09]
[1.85008128e+00 1.99515991e-08]
[2.61119257e-01 1.00523526e-09]
[4.12292850e-01 1.10308572e-09]
[1.39133613e+00 2.28679866e-09]
[9.85221964e+00 1.15318111e-13]
[4.25885713e+00 1.12965019e-08]
[1.13466152e+01 1.32374084e-13]
[2.56184600e+01 6.49157611e-08]
[3.75964598e+01 4.48889711e-07]
[3.25463599e+01 1.30611513e-07]
[3.0692762e+01 8.2397616e-13]
[8.70132654e+00 7.17945010e-13]
[3.51996721e+00 4.23538408e-14]
[3.21890177e+00 2.10932367e-11]
[4.62779985e+00 5.04657407e-13]
[6.13296365e+00 3.75545470e-08]
[9.60218527e+00 2.16901912e-08]
[4.31079654e-02 1.52155666e-10]
[1.27337165e+00 1.31538067e-13]
[9.12478566e-02 9.20088040e-14]
[6.58728123e+00 2.13150987e-12]
[9.84364815e-03 6.59033626e-15]
[1.10255721e-01 5.82163390e-14]
[1.92012707e-01 1.71105995e-13]
[2.83031104e-04 3.25431943e-16]
[1.23994005e+00 1.04697814e-12]
[6.84702203e-03 6.16529565e-15]
[1.78834466e-01 9.20402615e-14]
[2.0711141

In [34]:
# f_r

for index, row in full_df.iterrows():
    sigma_0, alpha_1, alpha_2, V_b, s = full_df['sigma_0'].iloc[index], full_df['alpha_1'].iloc[index], full_df['alpha_2'].iloc[index], full_df['V_b'].iloc[index], full_df['s'].iloc[index]
    f_r = V_b * (((1 - 10**(-(alpha_1 - 0.4) * V_b)) / ((alpha_1 - 0.4) * np.log(10))) + ((10**((alpha_2 - 0.4) * (20 - V_b)) - 1) / ((alpha_2 - 0.4) * np.log(10)))) ** (-1)
    print(f_r)
    
    
    
    
    

3.2006142917052647
2.053839249948675
0.7030645288154036
1.0150625037340049
0.4581686827581609
0.293348603883651
0.1907814327846603
0.4709632145662931
0.3124535359617094
0.492252603203343
0.8017793032583919
0.48119105084148744
0.42138559153360167
0.31462832301863586
4.839241437667308
0.28544379640308404
0.15086617103719752
0.4881606801587542
0.648438011339254
0.3027443328590394
0.4158533870945613
2.3514399058438165
0.6270537016727202
0.9124896582785091
0.165941760644814
0.3468044203205913
0.7207257539174958
0.07152743046136843
0.8619107580778205
0.18729943256093406
0.2733802822344534
0.6991142399428317
0.36103350526474626
0.298619214405689
0.14813760919641775
0.8477945388129523
0.08007570854397049
0.7942009298585807
0.6446530239323472
1.0309533561227635
0.5566284781464896
0.5378788163064948
0.48043408675347754
1.14178693998433
0.533348327033325
0.5147111883098605
0.5174134281038353
0.48757006813142384
0.3626173613287592
0.417528679673456
0.43304606784486555
0.31827372813633026
4.7805072

10^(0.4) term by Dnv/dv and integrate from all the magnitudes (1 - 20)

f_r

In [31]:
# f numbers
# one value of reference stars for each longitude and latitude

# SynthPop

print(theta_first)
print(sigma_0)

(305525.97733146325, 0.4871861542375649, 0.1010332845020712, 18.52323083372569, 9.941294488208666)
305525.97733146325
